In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import re
import geopandas as gpd
from shapely.geometry import Point

from datetime import datetime
import calendar

from tqdm.auto import tqdm

In [2]:
def convert_degrees(string: str) -> float:
    """
    """
    
    try:
        return float(string)
    except:
        aux = string.replace(' ', '')
        coords = re.sub(r'[°⁰o′″º°\'"¹I,ʼ]', ' ', aux).strip().split(' ')
        return np.sum([float(x) / 60**i for i, x in enumerate(coords[:3])])

In [3]:
PATH_KZ = Path('Z:/nahaUsers/casadje/datasets/Kazakhstan/reservoirs')

## Attributes

In [4]:
reservoirs = pd.read_excel(
    PATH_KZ / 'reservoirs.xlsx',
    sheet_name='processed',
    index_col=0
)

rename_cols = {
    'Name of the structures': 'RES_NAME',
    # 'Branch',
    # 'Class',
    # 'Water source',
    'Year of commissioning': 'YEAR',
    'total volume': 'CAP_MCM',
    # 'effective volume',
    # 'Type of regulation and purpose (long-term, seasonal, standard, liquid)',
    # 'Capacity of the facility, m3/s',
    # 'Total area, ha',
    # 'Mirror area, km2',
    # 'Percentage of wear ', 
    # 'Technical condition', 
    # 'Paradise center (km)',
    # 'The nearest settlement (km)',
    'longitude': 'LON',
    'latitude': 'LAT',
    'Maximum depth according to the project, m': 'DEPTH_M',
    # 'The year of acceptance on the balance sheet',
    # 'On the balance sheet (specify whose account)'
}
reservoirs.rename(columns=rename_cols, inplace=True)

In [5]:
# convert to shapefile
reservoirs.LAT = [convert_degrees(lat) for lat in reservoirs.LAT]
reservoirs.LON = [convert_degrees(lat) for lat in reservoirs.LON]
geometry = [Point(lon, lat) for lon, lat in zip(reservoirs.LON, reservoirs.LAT)]
reservoirs = gpd.GeoDataFrame(reservoirs, geometry=geometry).set_crs(epsg=4326)

In [ ]:
reservoirs.to_file(PATH_KZ / 'reservoirs.shp')

## Time series

In [176]:
def read_header(xls_file, sheet_name=None):
    
    header = pd.read_excel(file, sheet_name=sheet_name, nrows=2)

    years = []
    for col in header.columns:
        try:
            years.append(int(col))
        except:
            continue

    variables = list(set([col.lower() for col in header.iloc[0, :].values if isinstance(col, str)]))

    # name = header.loc[1, 'Name of the reservoir '].split()
    
    return years, variables#, name

In [177]:
file, res_name

(WindowsPath('Z:/nahaUsers/casadje/datasets/Kazakhstan/reservoirs/timeseries/raw/Northern Kazakhstan/Northern Kazakhstan.xlsx'),
 'Sergeyevsky')

In [195]:
    years, variables = read_header(file, sheet_name=res_name)

In [196]:
variables

['discharge of million m3', 'volume million m3', 'inflow of million m3']

In [209]:
    # import time series data
    data = pd.read_excel(file, sheet_name=res_name, skiprows=1).iloc[:, 2:]
    
    # remove empty days
    data = data[data.iloc[:, 1].notnull()]

    #  create index
    data.columns = ['month', 'day'] + data.columns.tolist()[2:]
    if data.shape[0] == 366:
        idx = pd.date_range(datetime(2000, 1, 1), datetime(2000, 12, 31))
    elif data.shape[0] == 365:
        idx = pd.date_range(datetime(2001, 1, 1), datetime(2001, 12, 31))
    else:
        raise ValueError(f"Unexpected shape of data: {data.shape}")
    data.month = idx.month
    data.day = idx.day
    data.set_index(['month', 'day'], inplace=True)

In [210]:
    # make sure that all values are float
    for col in data.columns:
        if data[col].dtype == 'object':
            try:
                data[col] = data[col].astype(float)
            except:
                data[col] = data[col].str.replace(',', '.')
                # data[col] = data[col].str.replace('..', '.')
                data[col].replace(to_replace=r'\s+', value=np.nan, regex=True, inplace=True)
                # data[col] = data[col].replace('', np.nan)
                data[col] = data[col].astype(float)

In [212]:
    # reorganise data
    dates = pd.date_range(start=datetime(years[0], 1, 1), end=datetime(years[-1], 12, 31))
    ts = pd.DataFrame(index=dates, columns=variables)
    ts.index.name = 'date'
    for y, (year) in enumerate(years):
        # select data from that year
        icols = list(y * len(variables) + np.arange(len(variables)))
        df = data.iloc[:, icols].copy()
        df.columns = variables
        df = df.astype(float)
        # remove February 29 if not leap year
        if not calendar.isleap(year) and (df.shape[0] == 366):
            df.drop((2, 29), axis=0, inplace=True)
        df.index = [datetime(year, month, day) for month, day in df.index]
        # save values
        ts.loc[df.index, variables] = df[variables].values  
        
    # keep period with data
    start, end = ts[variables].first_valid_index(), ts[variables].last_valid_index()
    ts = ts.loc[start:end]

In [241]:
def read_timeseries(xls_file, sheet_name=None):
    
    # read header
    years, variables = read_header(xls_file, sheet_name=sheet_name)
    
    # import time series data
    data = pd.read_excel(xls_file, sheet_name=sheet_name, skiprows=1).iloc[:, 2:]
    
    # remove empty days
    data = data[data.iloc[:, 1].notnull()]
    
    #  create index
    data.columns = ['month', 'day'] + data.columns.tolist()[2:]
    if data.shape[0] == 366:
        idx = pd.date_range(datetime(2000, 1, 1), datetime(2000, 12, 31))
    elif data.shape[0] == 365:
        idx = pd.date_range(datetime(2001, 1, 1), datetime(2001, 12, 31))
    else:
        raise ValueError(f"Unexpected shape of data: {data.shape}")
    data.month = idx.month
    data.day = idx.day
    data.set_index(['month', 'day'], inplace=True)
    
    # make sure that all values are float
    for col in data.columns:
        if data[col].dtype == 'object':
            try:
                data[col] = data[col].astype(float)
            except:
                data[col].replace(',+', '.', regex=True, inplace=True)
                # data[col] = data[col].str.replace('..', '.')
                data[col].replace('-', np.nan, inplace=True)
                data[col].replace('p', np.nan, inplace=True)
                data[col].replace(to_replace=r'\s+', value=np.nan, regex=True, inplace=True)
                data[col] = data[col].astype(float)
    
    # reorganise data
    dates = pd.date_range(start=datetime(years[0], 1, 1), end=datetime(years[-1], 12, 31))
    ts = pd.DataFrame(index=dates, columns=variables)
    ts.index.name = 'date'
    for y, (year) in enumerate(years):
        # select data from that year
        icols = list(y * len(variables) + np.arange(len(variables)))
        df = data.iloc[:, icols].copy()
        df.columns = variables
        df = df.astype(float)
        # remove February 29 if not leap year
        if not calendar.isleap(year) and (df.shape[0] == 366):
            df.drop((2, 29), axis=0, inplace=True)
        df.index = [datetime(year, month, day) for month, day in df.index]
        # save values
        ts.loc[df.index, variables] = df[variables].values         
    
    # keep period with data
    start, end = ts[variables].first_valid_index(), ts[variables].last_valid_index()
    ts = ts.loc[start:end]
    
    return ts

In [224]:
path_ts = PATH_KZ / 'timeseries'
basins = [item.stem for item in (path_ts / 'raw').iterdir() if item.is_dir()]

for basin in tqdm(basins, desc='basins'):
    path = path_ts / 'raw' / basin

basins:   0%|          | 0/13 [00:00<?, ?it/s]

In [248]:
path = path_ts / 'raw' / 'Zhetysu'

In [249]:
    files = path.glob('*.xlsx')
    for file in tqdm(files, desc='reservoirs'):
        data_dct = pd.read_excel(file, sheet_name=None)
        for res_name in data_dct.keys():

            # res_name = file.stem.split()[0]

            out_file = path_ts / 'processed' / f'{res_name.lower()}.csv'
            if out_file.is_file():
                continue
            print(res_name)

            # years, variables, name = read_header(file)
            ts = read_timeseries(file, sheet_name=res_name)
            ts.to_csv(out_file)

reservoirs: 0it [00:00, ?it/s]

2010-2017


ValueError: Unexpected shape of data: (97, 10)

In [ ]:
res_name